In [1]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        continue
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
       # print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

# 加载模型

In [2]:
import argparse
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [3]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
from rcan import make_model

In [5]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=[2]
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1

In [6]:
model = make_model(modelargs()).to(device)

# 定义psnr函数

In [7]:
def get_psnr(testdata,myscale,picidx):
   # print(filename)
    scale=myscale
   # text= os.path.split(filename)[-1]
  #  textname = os.path.splitext(text)[0]
    
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    
    clrpath=file_name(lrpath)
    chrpath=file_name(hrpath)
  #  print(clrpath[picidx])
    textname=str(clrpath[picidx]).split("_LRBI")[0]
#     textname=textnames[0]
#     for i in range(len(textnames)-1):
#         if(textnames[i+1]=='LRBI'):
#             break
#         textname=textname+textnames[i+1]
        
        
    
    
    
    lr = pil_image.open(lrpath+clrpath[picidx]).convert('RGB')
    lr = np.array(lr).astype(np.float32).transpose([2,0,1])
 #   lrycbcr = rgb2ycbcr(lr)
    
   # y = lrycbcr[..., 0]
   # print(lr.shape)
    y =lr /255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0)
    
    
    #print("y:{}".format(y.size()))########################################################################

    hr = pil_image.open(hrpath+chrpath[picidx]).convert('RGB')
    hr=np.array(hr).astype(np.float32).transpose([2,0,1])
  #  hrycbcrb = rgb2ycbcr(hr)
  #  ya = hrycbcrb[..., 0]
  
    ya =hr/ 255.
    ya = torch.from_numpy(ya).to(device)
    ya = ya.unsqueeze(0)
  #  print("ya:{}".format(ya.size()))########################################################################

    #print(y.size())

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(ya, preds)

    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0)
    
    #print(preds.shape)
    output = np.array(preds).transpose([1, 2, 0])
    output = np.clip((output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    savepath='./{}/x{}/{}_idn_x{}.png'.format(testdata,myscale,textname,scale)
    
    if not os.path.exists(os.path.split(savepath)[0]):
        os.makedirs(os.path.split(savepath)[0])
     
    
    output.save(savepath)
    
    preds = None
    output = None
    ya = None
    y = None
    lr = None
    hr = None
    
    
    
    return psnr


# 单独计算psnr

In [8]:
modelname='./checkpoint/model_epoch_989.pth'
print(modelname)

./checkpoint/model_epoch_989.pth


In [9]:
from model import IDN

In [10]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=2
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1
        self.d=16
        self.s=4

In [11]:
model = IDN(modelargs())

In [12]:

state_dict = model.state_dict()
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)


In [13]:
testdata="Set5"
myscale = 2

In [14]:
hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)

In [15]:
count=len(clrpath)

In [16]:


hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)

5


In [17]:

model=model.to(device)
model.eval()

IDN(
  (fblock): FBlock(
    (module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.05)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): LeakyReLU(negative_slope=0.05)
    )
  )
  (dblocks): Sequential(
    (0): DBlock(
      (enhancement_top): Sequential(
        (0): Conv2d(64, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.05)
        (2): Conv2d(48, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4)
        (3): LeakyReLU(negative_slope=0.05)
        (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): LeakyReLU(negative_slope=0.05)
      )
      (enhancement_bottom): Sequential(
        (0): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.05)
        (2): Conv2d(64, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [18]:

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 36.72
PSNR: 35.95
PSNR: 27.47
PSNR: 31.88
PSNR: 32.21
avg_psnr is :32.84


In [19]:
testdata="Manga109"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

109
PSNR: 32.31
PSNR: 29.05
PSNR: 38.39
PSNR: 33.67
PSNR: 30.64
PSNR: 32.97
PSNR: 29.68
PSNR: 29.84
PSNR: 29.84
PSNR: 26.69
PSNR: 29.72
PSNR: 29.51
PSNR: 31.65
PSNR: 30.43
PSNR: 30.57
PSNR: 29.23
PSNR: 30.21
PSNR: 34.81
PSNR: 32.25
PSNR: 32.60
PSNR: 31.38
PSNR: 31.10
PSNR: 34.67
PSNR: 32.20
PSNR: 28.72
PSNR: 31.81
PSNR: 35.38
PSNR: 33.00
PSNR: 33.53
PSNR: 32.07
PSNR: 27.84
PSNR: 27.85
PSNR: 33.73
PSNR: 29.55
PSNR: 28.63
PSNR: 32.18
PSNR: 27.02
PSNR: 33.49
PSNR: 35.24
PSNR: 27.36
PSNR: 30.54
PSNR: 33.13
PSNR: 22.44
PSNR: 35.16
PSNR: 24.41
PSNR: 29.72
PSNR: 26.95
PSNR: 34.14
PSNR: 26.75
PSNR: 29.62
PSNR: 29.26
PSNR: 33.03
PSNR: 39.02
PSNR: 34.00
PSNR: 24.45
PSNR: 34.26
PSNR: 30.46
PSNR: 31.18
PSNR: 24.04
PSNR: 32.63
PSNR: 32.09
PSNR: 30.29
PSNR: 34.29
PSNR: 34.40
PSNR: 32.53
PSNR: 24.23
PSNR: 31.53
PSNR: 29.19
PSNR: 32.98
PSNR: 30.21
PSNR: 28.68
PSNR: 34.63
PSNR: 34.53
PSNR: 30.60
PSNR: 29.98
PSNR: 31.16
PSNR: 31.30
PSNR: 26.65
PSNR: 31.24
PSNR: 32.24
PSNR: 30.62
PSNR: 33.30
PSNR: 31.05


In [20]:
testdata="Set14"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

14
PSNR: 22.96
PSNR: 26.84
PSNR: 27.32
PSNR: 28.94
PSNR: 25.72
PSNR: 31.86
PSNR: 29.38
PSNR: 32.18
PSNR: 33.00
PSNR: 28.72
PSNR: 32.89
PSNR: 30.78
PSNR: 26.62
PSNR: 30.88
avg_psnr is :29.15


In [21]:
testdata="B100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 25.08
PSNR: 29.69
PSNR: 27.48
PSNR: 32.88
PSNR: 28.31
PSNR: 34.88
PSNR: 29.39
PSNR: 24.99
PSNR: 29.89
PSNR: 31.72
PSNR: 25.85
PSNR: 30.98
PSNR: 33.42
PSNR: 30.64
PSNR: 27.15
PSNR: 28.40
PSNR: 35.46
PSNR: 34.00
PSNR: 26.47
PSNR: 27.14
PSNR: 24.14
PSNR: 25.86
PSNR: 25.94
PSNR: 28.22
PSNR: 30.09
PSNR: 29.77
PSNR: 29.87
PSNR: 32.36
PSNR: 32.06
PSNR: 21.22
PSNR: 32.17
PSNR: 22.55
PSNR: 28.16
PSNR: 25.69
PSNR: 35.59
PSNR: 28.93
PSNR: 28.58
PSNR: 29.07
PSNR: 30.94
PSNR: 35.31
PSNR: 28.15
PSNR: 26.78
PSNR: 27.97
PSNR: 29.47
PSNR: 25.85
PSNR: 35.62
PSNR: 24.90
PSNR: 27.16
PSNR: 28.95
PSNR: 33.46
PSNR: 26.38
PSNR: 25.14
PSNR: 33.50
PSNR: 34.02
PSNR: 29.93
PSNR: 27.23
PSNR: 22.42
PSNR: 28.79
PSNR: 33.36
PSNR: 31.58
PSNR: 32.95
PSNR: 30.30
PSNR: 34.96
PSNR: 26.51
PSNR: 27.61
PSNR: 30.55
PSNR: 39.49
PSNR: 23.28
PSNR: 24.01
PSNR: 30.79
PSNR: 30.88
PSNR: 27.70
PSNR: 29.16
PSNR: 27.65
PSNR: 25.50
PSNR: 32.04
PSNR: 25.43
PSNR: 31.84
PSNR: 31.46
PSNR: 31.69
PSNR: 37.22
PSNR: 29.33
PSNR: 23.80


In [22]:
testdata="Urban100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)


model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 28.55
PSNR: 27.33
PSNR: 25.37
PSNR: 23.26
PSNR: 25.65
PSNR: 24.01
PSNR: 28.19
PSNR: 23.22
PSNR: 33.15
PSNR: 27.49
PSNR: 17.96
PSNR: 25.45
PSNR: 28.31
PSNR: 26.56
PSNR: 26.85
PSNR: 28.36
PSNR: 26.14
PSNR: 27.29
PSNR: 22.30
PSNR: 23.15
PSNR: 30.04
PSNR: 29.22
PSNR: 28.58
PSNR: 20.69
PSNR: 29.48
PSNR: 29.29
PSNR: 30.47
PSNR: 31.30
PSNR: 26.09
PSNR: 24.05
PSNR: 26.18
PSNR: 28.99
PSNR: 27.62
PSNR: 24.31
PSNR: 26.67
PSNR: 28.09
PSNR: 25.57
PSNR: 28.02
PSNR: 25.00
PSNR: 22.18
PSNR: 23.35
PSNR: 27.35
PSNR: 25.11
PSNR: 27.92
PSNR: 23.53
PSNR: 26.16
PSNR: 23.51
PSNR: 21.47
PSNR: 23.30
PSNR: 26.62
PSNR: 27.27
PSNR: 26.67
PSNR: 23.40
PSNR: 23.54
PSNR: 26.53
PSNR: 25.06
PSNR: 28.76
PSNR: 24.82
PSNR: 24.53
PSNR: 24.66
PSNR: 25.82
PSNR: 22.66
PSNR: 20.39
PSNR: 27.74
PSNR: 26.21
PSNR: 25.34
PSNR: 19.79
PSNR: 29.51
PSNR: 26.02
PSNR: 26.40
PSNR: 29.04
PSNR: 19.78
PSNR: 21.85
PSNR: 25.30
PSNR: 29.48
PSNR: 23.48
PSNR: 26.21
PSNR: 28.43
PSNR: 26.91
PSNR: 34.98
PSNR: 29.13
PSNR: 29.46
PSNR: 24.11


In [13]:
myscale = 3
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
   
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 30.63
PSNR: 25.66
PSNR: 34.00
PSNR: 33.36
PSNR: 27.44
PSNR: 28.82
PSNR: 27.34
PSNR: 27.23
PSNR: 26.26


KeyboardInterrupt: 

In [ ]:
myscale = 4
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
      
    sum_psnr += get_psnr(testdata,myscale,i)
print("psnr is :{:.2f}".format(sum_psnr/count))